In [ ]:
import cudaq
import numpy as np
import json
import time


with open('qedc_hsim_data.json', 'r') as file:
    data = json.load(file)

h_x = data["h_x"]
h_z = data["h_z"]
K=10
#K = data["k"]
t = data ["t"]
w = data["w"]
    
    



@cudaq.kernel()
def HamiltonianSimulation(n_spins: int, K: int, t: float, w:int, h_x: list[float], h_z: list[float]):
    
    tau = t/K
    
    reg = cudaq.qvector(n_spins)
    
    for qubit in range(0, n_spins, 2):
        x(reg[qubit])
        
        
    for k in range(K):
        
        for qubit in range(n_spins):
            rx(2*tau*w*h_x[qubit], reg[qubit])
            rz(2*tau*w*h_z[qubit], reg[qubit])
        
        for j in range(2):
            for i in range(j % 2, n_spins - 1, 2):
                
                # Apply XXYYZZ Gate
                q0 = i
                q1 = (i+1)%n_spins
                
                rz(np.pi/2, reg[q1])
                cx(reg[q1], reg[q0])
                rz(np.pi* tau - np.pi/2, reg[q0])
                ry(np.pi/2 - np.pi*tau, reg[q1])
                cx(reg[q0], reg[q1])
                ry(np.pi*tau - np.pi/2, reg[q1])
                cx(reg[q1], reg[q0])
                rz(-np.pi/2, reg[q0])    

        
    

def run(n_qubits):
    
    hz = h_z[:n_qubits]
    hx = h_x[:n_qubits]
    
    start_time = time.time()
    result = cudaq.sample(HamiltonianSimulation, n_qubits, K, t, w, hx, hz, shots_count=1000000)
    end_time = time.time()
    
    print(result.most_probable())
    print(result.probability('10101010101010101010'))
    print(end_time-start_time)

run(28)